In [65]:
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [66]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [67]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog',
           'horse', 'ship', 'truck']

In [68]:
from keras.models import Sequential
from keras.layers import Dense, Rescaling, Flatten, AlphaDropout
from keras.backend import clear_session

clear_session()

model = Sequential()
model.add(Flatten(input_shape=[32, 32, 3]))
model.add(Rescaling(scale=1.0 / 255))

for _ in range(20):
    model.add(Dense(100, kernel_initializer="lecun_normal", activation="selu"))

model.add(AlphaDropout(rate=0.1))
model.add(Dense(10, activation="softmax"))

In [69]:
from keras.optimizer_v2.nadam import Nadam

nadam_opt = Nadam(learning_rate=5e-4)
model.compile(loss="categorical_crossentropy", optimizer=nadam_opt, metrics=["accuracy"])

In [70]:
import os
import time
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

root_logdir = os.path.join(os.curdir, "my_logs")
log_dir = os.path.join(root_logdir, time.strftime("run_%Y_%m_%d-%H_%M_%S"))

early_stopping = EarlyStopping(patience=20)
model_checkpoint = ModelCheckpoint("my_cifar10_model_resc_alpha.h5",
                                   save_best_only=True)
tensorboard = TensorBoard(log_dir)

history = model.fit(x_train, y_train, epochs=100,
                    validation_data=(x_val, y_val),
                    callbacks=[early_stopping, model_checkpoint, tensorboard])

Epoch 1/100
1172/1172 [==============================] - 6s 4ms/step - loss: 2.1017 - accuracy: 0.2127 - val_loss: 1.9278 - val_accuracy: 0.2986
Epoch 2/100
1172/1172 [==============================] - 5s 4ms/step - loss: 1.8830 - accuracy: 0.3123 - val_loss: 1.8145 - val_accuracy: 0.3394
Epoch 3/100
1172/1172 [==============================] - 5s 4ms/step - loss: 1.8103 - accuracy: 0.3476 - val_loss: 1.9605 - val_accuracy: 0.3423
Epoch 4/100
1172/1172 [==============================] - 5s 4ms/step - loss: 1.7581 - accuracy: 0.3693 - val_loss: 1.7212 - val_accuracy: 0.3857
Epoch 5/100
1172/1172 [==============================] - 5s 4ms/step - loss: 1.7143 - accuracy: 0.3865 - val_loss: 1.7872 - val_accuracy: 0.3874
Epoch 6/100
1172/1172 [==============================] - 5s 4ms/step - loss: 1.6704 - accuracy: 0.4018 - val_loss: 1.7005 - val_accuracy: 0.3877
Epoch 7/100
1172/1172 [==============================] - 5s 4ms/step - loss: 1.6340 - accuracy: 0.4182 - val_loss: 1.6333 - val_ac

In [71]:
from keras.models import load_model

model = load_model("my_cifar10_model_resc_alpha.h5")
model.evaluate(x_val, y_val)

391/391 [==============================] - 0s 1ms/step - loss: 1.5099 - accuracy: 0.4886


[1.5099198818206787, 0.4885599911212921]